## Multi-Agent Supervisor

Based on the example here: 

https://langchain-ai.github.io/langgraph/tutorials/multi_agent/agent_supervisor/#create-agent-supervisor



In [5]:
from typing import Literal
from typing_extensions import TypedDict
import sys

from langchain_anthropic import ChatAnthropic
from langgraph.graph import MessagesState, END
from langgraph.types import Command
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

members = ["researcher", "coder"]
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed
options = members + ["FINISH"]

system_prompt = (
    "You are a supervising investment manager. Your goal is to"
     "provide investment advice. You have access to the"
    f" following team members or  workers: {members}. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH. "
)


class Router(TypedDict):
    """Worker to route to next. If no workers needed, route to FINISH."""
    next: Literal[*options]


llm = ChatOpenAI(model_name="gpt-4o-mini")


class State(MessagesState):
    next: str


def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
    print('Starting the supervisor node with state:',state)
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"]
    
    response = llm.with_structured_output(Router).invoke(messages)
    print('The response from supervisor llm call is ', response)
    
    goto = response["next"]
    if goto == "FINISH":
        goto = END

    return Command(goto=goto, update={"next": goto})

In [6]:
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent
from ungoliant.tools import polygon_tools as pt

research_agent = create_react_agent(
    llm, tools=[pt.close_price_tool], state_modifier="You are a researcher. DO NOT do any math."
)

# This node takes the state and returns a command class which must contain supervisor. 
# This means that the research node can only only go back to the supervisor. 
def research_node(state: State) -> Command[Literal["supervisor"]]:
    result = research_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="researcher")
            ]
        },
        goto="supervisor",
    )


# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION, WHICH CAN BE UNSAFE WHEN NOT SANDBOXED
code_agent = create_react_agent(llm, tools=[pt.close_price_tool])


def code_node(state: State) -> Command[Literal["supervisor"]]:
    result = code_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="coder")
            ]
        },
        goto="supervisor",
    )


builder = StateGraph(State)
builder.add_edge(START, "supervisor")
builder.add_node("supervisor", supervisor_node)
builder.add_node("researcher", research_node)
builder.add_node("coder", code_node)
graph = builder.compile()

In [8]:
for s in graph.stream(
    {"messages": [("user", "What is the share price range of Apple on 1/14 of 2025?")]}, subgraphs=True
):
    print(s)
    #print("----")
    #print('')

Starting the supervisor node with state: {'messages': [HumanMessage(content='What is the share price range of Apple on 1/14 of 2025?', additional_kwargs={}, response_metadata={}, id='e5b648fc-c3ed-4480-9b9f-2bfc8d3fb808')]}
The response from supervisor llm call is  {'next': 'researcher'}
((), {'supervisor': {'next': 'researcher'}})
(('researcher:448288d6-6e8a-928f-8679-83ec831a74a1',), {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FmsA7bNHHMFxWHTRLIzPgWly', 'function': {'arguments': '{"ticker":"AAPL","date":"2025-01-14"}', 'name': 'get_close_price'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 113, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system